<a href="https://colab.research.google.com/github/RaswanthMurugan20/ML4SCI/blob/main/Graph_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash 
python -c "import torch; print(torch.__version__)"
python -c "import torch; print(torch.version.cuda)"
python -c "import torch; print(torch.cuda.is_available())"
export PATH=/usr/local/cuda/bin:$PATH
echo $PATH
export CPATH=/usr/local/cuda/include:$CPATH
echo $CPATH
export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH
echo $LD_LIBRARY_PATH

sudo -H pip config set global.cache-dir false
pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
pip install torch-geometric
pip install torch torchvision

1.10.0+cu111
11.1
False
/usr/local/cuda/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
/usr/local/cuda/include:
/usr/local/cuda/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
Writing to /root/.config/pip/pip.conf
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=722149722a7bf351ff8ac983f21cb8031ab1e7d782d5948b9b5a31b038df3528
  Stored in directory: /content/false/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
import numpy as np 
import torch_geometric
import torch_geometric.nn as geom_nn
import torch_geometric.data as geom_data
from torch_cluster import knn_graph
import math
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [3]:
CUDA_LAUNCH_BLOCKING = 1 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(device)

cpu


In [4]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/QG_jets_withbc_10.npz /content/

Mounted at /content/drive


In [5]:
data = np.load('/content/QG_jets_withbc_10.npz')
x, y = data[data.files[0]], data[data.files[1]]

In [6]:
"""
ADDITIONAL FEATURES

* q  -  electric charge of the particle 
* isElectron - if the particle is an electron 
* isMuon - if the particle is a muon
* isChargedHadron - if the particle is a charged hadron 
* isNeutralHadron - if the particle is a neutral hadron
* isPhoton - if the particle is a photon

reference : - https://arxiv.org/abs/1902.08570

The pdgid dictionary returns features according to the PDGID value 

"""  

pdgid = {
    -11: np.array([-1,1,0,0,0,0]), 
     11: np.array([1,1,0,0,0,0]),
     13: np.array([-1,0,1,0,0,0]),
    -13: np.array([1,0,1,0,0,0]), 
     22: np.array([0,0,0,0,0,1]),
    130: np.array([0,0,0,0,1,0]), 
    211: np.array([1,0,0,1,0,0]), 
   -211: np.array([-1,0,0,1,0,0]), 
    321: np.array([1,0,0,1,0,0]), 
   -321: np.array([-1,0,0,1,0,0]), 
   2112: np.array([0,0,0,0,1,0]), 
  -2112: np.array([0,0,0,0,1,0]), 
   2212: np.array([1,0,0,1,0,0]), 
  -2212: np.array([-1,0,0,1,0,0]), 
      0: np.array([0,0,0,0,0,0])
}

In [7]:
def FeatureAdd(input):
  new_features = np.concatenate([pdgid[a][np.newaxis,:] for a in input[:,3]], axis = 0)
  # input
  # np.concatenate((input,new_features),axis = 1)
  return input

# creating a dataset
def GnnDataset(a,b):
    dataset = []
    n = a.shape[1]
    for i in range(a.shape[0]):
      Jet = torch_geometric.data.data.Data()
      Jet.x = torch.tensor(FeatureAdd(a[i])).float()
      Jet.edge_index = knn_graph(torch.tensor(a[i]).float(), k = 16, loop=True) 
      Jet.y = torch.tensor(b[i]).float()
      Jet.num_nodes = n
      dataset.append(Jet.to(device))
    return dataset

In [8]:
dataset = GnnDataset(x,y)

In [9]:
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import global_max_pool

# Edge Convolution Layer

class ParticleNetLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        # Message passing with "max" aggregation.
        super().__init__(aggr='max')
        
        self.mlp = Sequential(Linear(in_channels + 2, out_channels),
                              ReLU(),
                              Linear(out_channels, out_channels))
        
    def forward(self, x, edge_index):
        # Start propagating messages.
        return self.propagate(edge_index, x=x)
    
    def message(self, x_j, x_i):
        # pos is used to compute the distance between the nodes, 
        # it uses rapidity and azimuthal angle of each particle in the jet 
        # for computing the distance

        pos = x_i[:,1:3] - x_j[:,1:3]
        input = torch.cat([x_i, pos], dim=1) 

        return self.mlp(input)  # Apply our final MLP.

In [10]:
class ParticleNet(torch.nn.Module):
    def __init__(self):
        super().__init__()

        np.random.seed(0)
        self.Edgeconv1 = ParticleNetLayer(4, 32)
        self.Edgeconv2 = ParticleNetLayer(32, 64)
        self.Linear = Linear(64, 1)
        self.Sigmoid = nn.Sigmoid()

    def forward(self, data, batch):

        edge_index = data.edge_index
        x = data.x

        # Compute the kNN graph:
        # Here, we need to pass the batch vector to the function call in order
        # to prevent creating edges between points of different examples.
        # We also add `loop=True` which will add self-loops to the graph in
        # order to preserve central point information.
        
        x = self.Edgeconv1(x=x, edge_index=edge_index)
        x = x.relu()
        x = self.Edgeconv2(x=x, edge_index=edge_index)
        x = x.relu()

        # # 4. Global Pooling.
        x = global_max_pool(x, batch)  # [num_examples, hidden_channels]

        # 5. Classifier.
        return self.Sigmoid(self.Linear(x))


model = ParticleNet()
print(model)

ParticleNet(
  (Edgeconv1): ParticleNetLayer()
  (Edgeconv2): ParticleNetLayer()
  (Linear): Linear(in_features=64, out_features=1, bias=True)
  (Sigmoid): Sigmoid()
)


In [11]:
Gconv = geom_nn.GCNConv

# This is a 2-layer Graph Convolutional Networks (GCN)
class GCNNet(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.Graphconv1 = Gconv(in_channels = 4, out_channels = 32)
        self.Graphconv2 = Gconv(in_channels = 32, out_channels = 64)
        self.Linear = Linear(64, 1)
        self.Sigmoid = nn.Sigmoid()

    def forward(self, data, batch):
        edge_index = data.edge_index
        x = data.x

        x = self.Graphconv1(x=x, edge_index=edge_index)
        x = x.relu()
        x = self.Graphconv2(x=x, edge_index=edge_index)
        x = x.relu()

        # # 4. Global Pooling.
        x = global_max_pool(x, batch)  # [num_examples, hidden_channels]
        
        # 5. Classifier.
        return self.Sigmoid(self.Linear(x))


model = GCNNet()
print(model)

GCNNet(
  (Graphconv1): GCNConv(4, 32)
  (Graphconv2): GCNConv(32, 64)
  (Linear): Linear(in_features=64, out_features=1, bias=True)
  (Sigmoid): Sigmoid()
)


In [12]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
from sklearn.metrics import roc_auc_score
from torch_geometric.loader import DataLoader
 

# splitting data into training and testing 70 - 30
train_loader = DataLoader(dataset[:70000], batch_size=10, shuffle=True)
test_loader = DataLoader(dataset[70000:], batch_size=10)


lossplt = []
accplt = []
epochs = 10

def train(model, optimizer, loader, criterion):
    model.train()
    total_loss = 0

    for epoch,data in enumerate(loader):
        optimizer.zero_grad()  # Clear gradients.
        logits = model(data, data.batch).reshape(-1)  # Forward pass.
        loss = criterion(logits, data.y)  # Loss computation.
        loss.backward()  # Backward pass.
        optimizer.step()  # Update model parameters.
        total_loss += loss.item() * data.num_graphs

    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(model, loader):
    model.eval()
    total_correct = 0
    batches = 0
    for data in loader:
        logits = model(data, data.batch)
        pred = logits > 0.5
        total_correct += int((pred == data.y.reshape(-1,1)).sum())

    return total_correct / len(loader.dataset)


def model_eval(model,train_loader,test_loader,epochs):

  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
  criterion = nn.BCELoss()
  
  model.to(device)
  lossplt = []
  accplt = []

  test_acc = test(model, test_loader)
  accplt.append(test_acc)
  
  print(f'Epoch: 0, Test Accuracy: {test_acc:.4f}')
  for epoch in range(1, epochs):
      loss = train(model, optimizer, train_loader, criterion)
      test_acc = test(model, test_loader)
      print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Test Accuracy: {test_acc:.4f}')
      lossplt.append(loss)
      accplt.append(test_acc)

  return lossplt, accplt

<IPython.core.display.Javascript object>

In [ ]:
epochs = 10

print("ParticleNet")
ParticleNetloss, ParticleNetAcc = model_eval(ParticleNet(),train_loader,test_loader,epochs)

print("Graph Convolutional Network")
GCNloss, GCNAcc = model_eval(GCNNet(),train_loader,test_loader,epochs)

ParticleNet
Epoch: 0, Test Accuracy: 0.5043
Epoch: 01, Loss: 0.5672, Test Accuracy: 0.7321


In [ ]:
fig = plt.figure()
fig.suptitle('Loss vs Epoch', fontsize=20)
plt.xlabel('Epochs', fontsize=18)
plt.ylabel('Loss', fontsize=16)
plt.plot(np.arange(epochs-1),ParticleNetloss, 'r', label='ParticleNet')
plt.legend()
plt.plot(np.arange(epochs-1),GCNloss,'b', label='GCN')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure()
fig.suptitle('Accuracy vs Epoch', fontsize=20)
plt.xlabel('Epochs', fontsize=18)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(np.arange(epochs),ParticleNetAcc, 'r', label='ParticleNet')
plt.legend()
plt.plot(np.arange(epochs),GCNAcc,'b', label='GCN')
plt.legend()
plt.show()